In [3]:
import torch
from going_modular import going_modular
from pathlib import Path
import torch
import torchvision
import torchinfo
from torch import nn
from torchvision import transforms

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
class DCGAN(nn.Module):
    def __init__(
        self,
        
    ):
        
        super().__init__()
        
        
        
        
    def forward(x):